In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torch.optim import Adam
import torchvision.transforms.functional as fn

import os
import time
import math
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io

from di_dataset2 import DepthImageDataset, collate_batch

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Hyperparams

In [ ]:
latent_dim = 200
num_epochs = 20
batch_size = 32
learning_rate = 1e-3

save_model = True
load_model = False

save_model_file = "../../../vae_models/vae_resnet_full.pth"
load_model_file = "../../../vae_models/vae_resnet.pth"

# Load Dataset

In [ ]:
# saves_folders = "../../../rl_data"
# load_paths = [os.path.join(saves_folders, saves_folder) for saves_folder in os.listdir(saves_folders)]
# load_path = "../../../rl_data/saves_2"

# # Obtain train and test data
# dataset = DepthImageDataset(load_path=load_path)

# train_size = int(0.9 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=0)
# test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=0)
# len(train_loader), len(test_loader), len(train_dataset), len(test_dataset)

In [ ]:
tfrecord_folder='../../../rl_data/tfrecord'
tfrecord_test_folder='../../../rl_data/tfrecord/test'

In [ ]:
train_dataset = DepthImageDataset(tfrecord_folder=tfrecord_folder, batch_size=batch_size) # 176 tfrecords
test_dataset = DepthImageDataset(tfrecord_folder=tfrecord_test_folder, batch_size=batch_size) # 20 tfrecords

In [ ]:
# len_train_dataset = sum(1 for _ in train_dataset) # 11223 batches (of 32),\
# len_test_dataset = sum(1 for _ in test_dataset) # 1305 batches (of 32)
# len_train_dataset, len_test_dataset = 11223, 1305
len_train_dataset, len_test_dataset = (65, 80) # 1 tfrecord will have varying lengths
len_train_dataset, len_test_dataset 

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=1, collate_fn=collate_batch)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, collate_fn=collate_batch)

In [ ]:
dataiter = iter(train_loader)
image_batch, *_ = dataiter.next() # image, height, width, depth
image_batch.shape

In [ ]:
def imshow(image):
    io.imshow(image.squeeze().numpy())
    io.show()

for image in image_batch[0:4]:
    imshow(image)
    
image.shape

In [ ]:
image_batch.squeeze().mean(), image_batch.squeeze().var()

# Define Variational Autoencoder

Adapted from https://github.com/julianstastny/VAE-ResNet18-PyTorch

In [ ]:
class ResizeConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, scale_factor, mode='nearest'):
        super().__init__()
        self.scale_factor = scale_factor
        self.mode = mode
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=1)

    def forward(self, x):
        x = F.interpolate(x, scale_factor=self.scale_factor, mode=self.mode)
        x = self.conv(x)
        return x

class BasicBlockEnc(nn.Module):

    def __init__(self, in_planes, stride=1):
        super().__init__()

        planes = in_planes*stride

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        if stride == 1:
            self.shortcut = nn.Sequential()
        else:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class BasicBlockDec(nn.Module):

    def __init__(self, in_planes, stride=1):
        super().__init__()

        planes = int(in_planes/stride)

        self.conv2 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(in_planes)
        # self.bn1 could have been placed here, but that messes up the order of the layers when printing the class

        if stride == 1:
            self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
            self.bn1 = nn.BatchNorm2d(planes)
            self.shortcut = nn.Sequential()
        else:
            self.conv1 = ResizeConv2d(in_planes, planes, kernel_size=3, scale_factor=stride)
            self.bn1 = nn.BatchNorm2d(planes)
            self.shortcut = nn.Sequential(
                ResizeConv2d(in_planes, planes, kernel_size=3, scale_factor=stride),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn2(self.conv2(x)))
        out = self.bn1(self.conv1(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ResNet18Enc(nn.Module):

    def __init__(self, num_Blocks=[2,2,2,2], z_dim=10, nc=3):
        super().__init__()
        self.in_planes = 64
        self.z_dim = z_dim
        self.conv1 = nn.Conv2d(nc, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(BasicBlockEnc, 64, num_Blocks[0], stride=1)
        self.layer2 = self._make_layer(BasicBlockEnc, 128, num_Blocks[1], stride=2)
        self.layer3 = self._make_layer(BasicBlockEnc, 256, num_Blocks[2], stride=2)
        self.layer4 = self._make_layer(BasicBlockEnc, 512, num_Blocks[3], stride=2)
        self.linear = nn.Linear(512*2*4, 2 * z_dim)

    def _make_layer(self, BasicBlockEnc, planes, num_Blocks, stride):
        strides = [stride] + [1]*(num_Blocks-1)
        layers = []
        for stride in strides:
            layers += [BasicBlockEnc(self.in_planes, stride)]
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        #print(x.shape)
        x = self.layer1(x)
        #print(x.shape)
        x = self.layer2(x)
        #print(x.shape)
        x = self.layer3(x)
        #print(x.shape)
        x = self.layer4(x)
        #print(f"x_enc = {x.shape}")
        x = F.adaptive_avg_pool2d(x, (2,4))
        #print(f"x_enc = {x.shape}")
        x = x.view(x.size(0), -1)
        #print(f"x_enc = {x.shape}")
        x = self.linear(x)
        mu = x[:, :self.z_dim]
        logvar = x[:, self.z_dim:]
        return mu, logvar

class ResNet18Dec(nn.Module):

    def __init__(self, num_Blocks=[2,2,2,2], z_dim=10, nc=3):
        super().__init__()
        self.in_planes = 512
        self.nc = nc

        self.linear = nn.Linear(z_dim, 512*2*4)

        self.layer4 = self._make_layer(BasicBlockDec, 256, num_Blocks[3], stride=2)
        self.layer3 = self._make_layer(BasicBlockDec, 128, num_Blocks[2], stride=2)
        self.layer2 = self._make_layer(BasicBlockDec, 64, num_Blocks[1], stride=2)
        self.layer1 = self._make_layer(BasicBlockDec, 64, num_Blocks[0], stride=1)
        self.conv1 = ResizeConv2d(64, nc, kernel_size=3, scale_factor=2)

    def _make_layer(self, BasicBlockDec, planes, num_Blocks, stride):
        strides = [stride] + [1]*(num_Blocks-1)
        layers = []
        for stride in reversed(strides):
            layers += [BasicBlockDec(self.in_planes, stride)]
        self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, z):
        x = self.linear(z)
        x = x.view(z.size(0), 512, 2, 4)
        #print("x before interpoloate:", x.shape)
        #x = F.interpolate(x, size=[17, 30], mode='bilinear', align_corners=True)
        x = F.interpolate(x, size=[17, 30], mode='nearest')
        #print(x.shape)
        x = self.layer4(x)
        #print(x.shape)
        x = self.layer3(x)
        #print(x.shape)
        x = self.layer2(x)
        #print(x.shape)
        x = self.layer1(x)
        #print("fix layer 1")
        #print(x.shape)
        x = torch.sigmoid(self.conv1(x))
        x = x.view(x.size(0), self.nc, 272, 480)
        x = x[:,:,:270,:]
        #print(x.shape)
        return x

class VAE(nn.Module):

    def __init__(self, z_dim, nc=1):
        super().__init__()
        self.encoder = ResNet18Enc(z_dim=z_dim, nc=nc)
        self.decoder = ResNet18Dec(z_dim=z_dim, nc=nc)
    
        
    def forward(self, x):
        mean, logvar = self.encoder(x)
        z = self.reparameterize(mean, logvar)
        #print(f"z_dim = {z.shape}")
        x_hat = self.decoder(z)

        return mean, logvar, x_hat
    
    @staticmethod
    def reparameterize(mean, logvar):
        std = torch.exp(logvar / 2) # in log-space, sqrt is divide by two
        epsilon = torch.randn_like(std)
        return epsilon * std + mean



### Define Model

In [ ]:
vae_model = VAE(z_dim=latent_dim)

if load_model:
    vae_model.load_state_dict(torch.load(load_model_file))
    vae_model.eval()
else:
    vae_model.train()

optimiser = torch.optim.Adam(vae_model.parameters(), lr=learning_rate)

In [ ]:
from torchsummary import summary
summary(vae_model, (1, 270, 480))

# Training

In [ ]:
def loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.BCELoss(reduction='sum')(x_hat, x)
    KLD      = 0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())

    return reproduction_loss - KLD

In [ ]:
n_training_samples = len_train_dataset * 32 # 32 samples per batch
n_iterations = len_train_dataset
n_training_samples, n_iterations

In [ ]:
time_iteration = []
for epoch in range(num_epochs):
    overall_loss = 0
    overall_elbo = 0
    since = time.time()
    
    train_dataset = DepthImageDataset(tfrecord_folder=tfrecord_folder, batch_size=batch_size) # 176 tfrecords
    train_loader = DataLoader(dataset=train_dataset, batch_size=1, collate_fn=collate_batch)
    
    for i, (image, *_) in enumerate(train_loader):
        since_iter = time.time()

        optimiser.zero_grad()

        # VAE forward pass
        mu, logvar, x_hat = vae_model(image)

        # Loss
        loss = loss_function(image, x_hat, mu, logvar)

        overall_loss += loss

        # Update weights
        
        loss.backward()
        optimiser.step()
        
        
        time_iteration.append(time.time() - since_iter)
        iter_time_mean = np.array(time_iteration).mean()
        
        if (i+1) % 1 == 0:
            time_elapsed = time.time() - since
            print(f"Epoch: {epoch+1}/{num_epochs}, Step: {i+1}/{n_iterations}, Avg loss: {overall_loss/((i+1)*batch_size):.3f}, time: {time_elapsed:.2f}, Avg. per iter {iter_time_mean:.2f}, Est. time left {iter_time_mean*(n_iterations - (i+1)):.2f}")
        
    if (i+1) % 10 == 0:
        imshow(image[0])
        imshow(x_hat[0].detach())
            

In [ ]:
if save_model:
    torch.save(vae_model.state_dict(), save_model_file)

# Evaluation

In [ ]:
vae_model.eval()

# Visualise sample of images
with torch.no_grad():
#     for images, *_ in test_loader:
#         # VAE forward pass
#         _, _, x_hat = vae_model(images)

#         print(images.mean(), x_hat.mean())
#         print(images.var(), x_hat.var())
#         print(images.shape)
#         for idx in range(len(images)):
#             # show images
#             imshow(images[idx])
#             imshow(x_hat[idx])

    images, *_ = next(iter(test_loader))
    # VAE forward pass
    _, _, x_hat = vae_model(images)

    print(images.mean(), x_hat.mean())
    print(images.var(), x_hat.var())
    print(images.shape)
    for idx in range(len(images)):
        # show images
        imshow(images[idx])
        imshow(x_hat[idx])

In [ ]:
vae_model.eval()
# Visualise sample of images

with torch.no_grad():
#     for (images, *_) in train_loader:
#         # VAE forward pass
#         _, _, x_hat = vae_model(images)

#         print(images.mean(), x_hat.mean())
#         print(images.var(), x_hat.var())
#         print(images.shape)
#         for idx in range(len(images)):
#             # show images
#             imshow(images[idx])
#             imshow(x_hat[idx])
        
    images, *_ = next(iter(train_loader))
    # VAE forward pass
    _, _, x_hat = vae_model(images)

    print(images.mean(), x_hat.mean())
    print(images.var(), x_hat.var())
    print(images.shape)
    for idx in range(len(images)):
        # show images
        imshow(images[idx])
        imshow(x_hat[idx])
